In [1]:
def koblitz_encode(E, string):
    A = E.a4()
    B = E.a6()
    p = E.base_field().cardinality()
    b = 2^8
    m = str(string)
    n = len(m)
    m = sum(ord(m[k]) * b^k for k in range(n))
    d = 100
    for j in range(d):
        x = (d * m + j) % p
        s = (x^3 + A * x + B) % p
        if s == power_mod(s, int((p+1)/2), p):
            y = power_mod(s, int((p+1)/4), p)
            return (E([x,y]))
        
def koblitz_decode(P):
    b = 2^8
    d = 100
    lst = []
    m = int((floor((P[0]/P[2])))/d)
#     print(m)
    while m != 0:
        lst.append(chr(m % b))
        m //= b
        m = int(m)
#         print(m)
    return ''.join(lst)

In [2]:
p = 2^521 - 1
# proof.arithmetic(False)
F = GF(p)
A = p-3
B = 1093849038073734274511112390766805569936207598951683748994586394495953116150735016013708737573759623248592132296706313309438452531591012912142327488478985984
q = 6864797660130609714981900799081393217269435300143305409394463459185543183397655394245057746333217197532963996371363321113864768612440380340372808892707005449

E = EllipticCurve([F(A), F(B)])

In [3]:
koblitz_decode(koblitz_encode(E, 'criptografia uhul!'))

'criptografia uhul!'

In [4]:
def generate_keys(E):
    N = E.cardinality()
    print(N)
    a = ZZ.random_element(N)
    while gcd(a, N) != 1:
        print(a)
        a = ZZ.random_element(N)
    return [a, power_mod(a, -1, N)]

In [5]:
%%time
alice_keys = generate_keys(E)

6864797660130609714981900799081393217269435300143305409394463459185543183397655394245057746333217197532963996371363321113864768612440380340372808892707005449
CPU times: user 4min 22s, sys: 125 ms, total: 4min 22s
Wall time: 4min 23s


In [6]:
%%time
bob_keys = generate_keys(E)

6864797660130609714981900799081393217269435300143305409394463459185543183397655394245057746333217197532963996371363321113864768612440380340372808892707005449
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 131 µs


In [7]:
def base_point(E):
    P = E.random_point()
    while P.order() != E.cardinality():
        P = E.random_point()
        print(P)
    return P

In [8]:
%%time
P = base_point(E)

CPU times: user 828 ms, sys: 0 ns, total: 828 ms
Wall time: 841 ms


In [9]:
message = 'oi bob tudo bem?'

In [10]:
PM = koblitz_encode(E, message)

In [11]:
bob_PM = bob_keys[1] * PM

In [12]:
koblitz_decode(bob_PM)

'\xde2(\x8b\x01\x0bC\xcf\x91\x10\xae\x86*\xf1\x9e\x82\x83y\x1f4\x9e\x12\xd7J\xfdQ\r\x8b\x198\xf4\xc5\xf3n\xb7\x94\x80\xd4c\xe1\xd6\x05Zt\xefS\xb5Z\xf9\x90\xfe\xf1\xf2\xa2\xe9&DZ\x9b\x14\x06|\x12\xc8\x02'

In [13]:
koblitz_decode(bob_PM * bob_keys[0]), koblitz_decode(bob_PM), koblitz_decode(PM)

('oi bob tudo bem?',
 '\xde2(\x8b\x01\x0bC\xcf\x91\x10\xae\x86*\xf1\x9e\x82\x83y\x1f4\x9e\x12\xd7J\xfdQ\r\x8b\x198\xf4\xc5\xf3n\xb7\x94\x80\xd4c\xe1\xd6\x05Zt\xefS\xb5Z\xf9\x90\xfe\xf1\xf2\xa2\xe9&DZ\x9b\x14\x06|\x12\xc8\x02',
 'oi bob tudo bem?')

In [14]:
from playsound import playsound

ImportError: No module named playsound

In [ ]:
# !pip3 install playsound

In [15]:
from playsound import playsound

ImportError: No module named playsound